# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import time
import logging
import numpy as np
import pandas as pd

## Utils

In [2]:
def get_logger(level=logging.WARNING, force_level=False, name='logger'):
    
    logger_levels = [logging.CRITICAL, logging.ERROR, logging.WARNING,
                     logging.INFO, logging.DEBUG]
    
    if level not in logger_levels:
        raise ValueError('Invalid level name. Valid names {}'.format(logger_levels))
    
    formatter = 'UDACITY ARVATO PROJECT: %(asctime)s - %(levelname)s: %(message)s'
    date_format = '%d/%m/%Y %H:%M:%S'
    logging.basicConfig(format=formatter, datefmt=date_format, level=level)

    logger = logging.getLogger(name)
    if logger.getEffectiveLevel() == 0 or force_level:
        logger.setLevel(level)

    return logger


def set_log_level(verbose, debug):
    if verbose:
        get_logger(logging.INFO, force_level=True)
    if debug:
        get_logger(logging.DEBUG, force_level=True)


In [3]:
class Timer:
    
    def __init__(self, decimal_round=5):
        self.decimal_round = decimal_round

    def start(self):
        self.timer_start = time.perf_counter()
        self.timer_last = self.timer_start

    def lapse(self):
        last_lapse = self.timer_last
        self.timer_last = time.perf_counter()
        return round(self.timer_last - last_lapse, 5) 

    def total(self):
        return round(time.perf_counter() - self.timer_start, 5) 


In [4]:
def print_df_full(data):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)
    
    display(data)
    
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 10)
    pd.set_option('display.max_colwidth', 50)


def convert_to_numeric(string):
    try: 
        return float(string)
    except ValueError:
        return string


## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [5]:
# load in the data
#azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
#customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

azdias = pd.read_csv(r'./datasets/azdias.csv', index_col='Unnamed: 0')
customers = pd.read_csv(r'./datasets/customers.csv', index_col='Unnamed: 0')

C:\Users\magonzac\Anaconda3\envs\pruebas\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
logger = get_logger()
timer = Timer(decimal_round=5)

### 0.1. Check dataset integrity

#### Verify that the datasets are as shown in the above description

In [7]:
print('AZDIAS shape -', azdias.shape)
print('CUSTOMERS shape -', customers.shape)

AZDIAS shape - (891221, 366)
CUSTOMERS shape - (191652, 369)


In [8]:
demographic_columns = azdias.columns
extra_columns = [col for col in customers.columns if col not in demographic_columns]

print('Extra columns:', extra_columns)

Extra columns: ['PRODUCT_GROUP', 'CUSTOMER_GROUP', 'ONLINE_PURCHASE']


The dimensions and the extra columns, match the description.

In [9]:
azdias.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


In [10]:
customers[demographic_columns].head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,9626,2,1.0,10.0,NaN,NaN,NaN,NaN,10.0,1.0,...,3.0,5.0,3.0,2.0,6.0,9.0,7.0,3,1,4
1,9628,-1,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.0,6.0,3.0,0.0,9.0,NaN,3,1,4
2,143872,-1,1.0,6.0,NaN,NaN,NaN,NaN,0.0,1.0,...,4.0,10.0,13.0,11.0,6.0,9.0,2.0,3,2,4
3,143873,1,1.0,8.0,NaN,NaN,NaN,NaN,8.0,0.0,...,2.0,6.0,4.0,2.0,NaN,9.0,7.0,1,1,4
4,143874,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,7.0,...,4.0,3.0,5.0,4.0,2.0,9.0,3.0,1,1,3


In [11]:
customers[extra_columns].head()

,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE
0,COSMETIC_AND_FOOD,MULTI_BUYER,0
1,FOOD,SINGLE_BUYER,0
2,COSMETIC_AND_FOOD,MULTI_BUYER,0
3,COSMETIC,MULTI_BUYER,0
4,FOOD,MULTI_BUYER,0


#### Solving WARNINGS

We can look at the datatypes of columns 18 and 19 since we got a warning while we loaded the data.

In [12]:
warning_columns = azdias.columns[[18, 19]]
print('WARNING columns:', warning_columns)

WARNING columns: Index(['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'], dtype='object')


In [13]:
azdias[warning_columns].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891221 entries, 0 to 891220
Data columns (total 2 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   CAMEO_DEUG_2015  792242 non-null  object
 1   CAMEO_INTL_2015  792242 non-null  object
dtypes: object(2)
memory usage: 20.4+ MB


In [14]:
azdias['CAMEO_DEUG_2015'].unique()

array([nan, 8.0, 4.0, 2.0, 6.0, 1.0, 9.0, 5.0, 7.0, 3.0, '4', '3', '7',
       '2', '8', '9', '6', '5', '1', 'X'], dtype=object)

In [15]:
azdias['CAMEO_INTL_2015'].unique()

array([nan, 51.0, 24.0, 12.0, 43.0, 54.0, 22.0, 14.0, 13.0, 15.0, 33.0,
       41.0, 34.0, 55.0, 25.0, 23.0, 31.0, 52.0, 35.0, 45.0, 44.0, 32.0,
       '22', '24', '41', '12', '54', '51', '44', '35', '23', '25', '14',
       '34', '52', '55', '31', '32', '15', '13', '43', '33', '45', 'XX'],
      dtype=object)

We have 'X' and 'XX' as values in these columns which have not been given in the description, also there are 'nan' values.

In [16]:
def clean_warning_columns(df, columns, verbose=False, debug=False):
    set_log_level(verbose, debug)
    timer.start()
    
    logger.info('Start cleaning warning columns')
    
    logger.debug('Replacing "X", "XX" and "nan" values with -1')
    replacements = {"X": np.nan, "XX": np.nan}
    df[columns] = df[columns].replace(replacements)
    logger.debug('Replaced columns: {0} in {1} seconds'.format(list(columns), timer.lapse()))
    
    logger.debug('Changing data type to float')
    df[columns] = df[columns].astype(float)
    logger.debug('Changed data type to float in {} seconds'.format(timer.lapse()))
    
    logger.info('End cleaning warning columns in {} seconds'.format(timer.total()))
    return df

In [17]:
azdias = clean_warning_columns(azdias, warning_columns, verbose=True, debug=True)

UDACITY ARVATO PROJECT: 06/05/2021 08:20:39 - INFO: Start cleaning warning columns
UDACITY ARVATO PROJECT: 06/05/2021 08:20:39 - DEBUG: Replacing "X", "XX" and "nan" values with -1
UDACITY ARVATO PROJECT: 06/05/2021 08:20:40 - DEBUG: Replaced columns: ['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'] in 0.65941 seconds
UDACITY ARVATO PROJECT: 06/05/2021 08:20:40 - DEBUG: Changing data type to float
UDACITY ARVATO PROJECT: 06/05/2021 08:20:40 - DEBUG: Changed data type to float in 0.30519 seconds
UDACITY ARVATO PROJECT: 06/05/2021 08:20:40 - INFO: End cleaning warning columns in 0.97454 seconds


In [18]:
customers = clean_warning_columns(customers, warning_columns, verbose=True, debug=True)

UDACITY ARVATO PROJECT: 06/05/2021 08:20:40 - INFO: Start cleaning warning columns
UDACITY ARVATO PROJECT: 06/05/2021 08:20:40 - DEBUG: Replacing "X", "XX" and "nan" values with -1
UDACITY ARVATO PROJECT: 06/05/2021 08:20:40 - DEBUG: Replaced columns: ['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'] in 0.11463 seconds
UDACITY ARVATO PROJECT: 06/05/2021 08:20:40 - DEBUG: Changing data type to float
UDACITY ARVATO PROJECT: 06/05/2021 08:20:40 - DEBUG: Changed data type to float in 0.05907 seconds
UDACITY ARVATO PROJECT: 06/05/2021 08:20:40 - INFO: End cleaning warning columns in 0.17451 seconds


Check whether the replacement has been carried out properly.

In [19]:
azdias['CAMEO_DEUG_2015'].unique()

array([nan,  8.,  4.,  2.,  6.,  1.,  9.,  5.,  7.,  3.])

In [20]:
azdias['CAMEO_INTL_2015'].unique()

array([nan, 51., 24., 12., 43., 54., 22., 14., 13., 15., 33., 41., 34.,
       55., 25., 23., 31., 52., 35., 45., 44., 32.])

### 0.2. Explore metadata datasets

Additionally we have been given two excel books.

- DIAS Attributes - Values 2017.xlsx - Has a description of demographic columns, information about range of values each column can take and their respective meanings.Loaded in metadata_values.
- DIAS Information Levels - Attributes 2017.xlsx - Has detailed information about each column with additional notes wherever required. Loaded in metadata_info_levels.

We can use these two dataframes to understand the data.

In [21]:
metadata_values = pd.read_csv(r'./datasets/metadata/metadata_values.txt', sep='\t', encoding='UTF-16')
metadata_info_levels = pd.read_csv(r'./datasets/metadata/metadata_info_levels.txt', sep='\t', encoding='UTF-16')

In [22]:
metadata_values.head(10)

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,NaN,NaN,0,no classification possible
2,NaN,NaN,1,passive elderly
3,NaN,NaN,2,cultural elderly
4,NaN,NaN,3,experience-driven elderly
5,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,NaN,NaN,1,< 30 years
7,NaN,NaN,2,30 - 45 years
8,NaN,NaN,3,46 - 60 years
9,NaN,NaN,4,> 60 years


In [23]:
metadata_info_levels.head(10)

,Information level,Attribute,Description,Additional notes
0,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,NaN,ANREDE_KZ,gender,NaN
3,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...
5,NaN,FINANZ_SPARER,financial typology: money saver,NaN
6,NaN,FINANZ_VORSORGER,financial typology: be prepared,NaN
7,NaN,FINANZ_ANLEGER,financial typology: investor,NaN
8,NaN,FINANZ_UNAUFFAELLIGER,financial typology: unremarkable,NaN
9,NaN,FINANZ_HAUSBAUER,financial typology: main focus is the own house,NaN


##### Metadata datasets preprocessing

We can use the ffill() method of pandas, to replace the NaN values of the dataset by the value of the previous row.

In [24]:
metadata_values[['Attribute', 'Description']] = metadata_values[['Attribute', 'Description']].ffill()

In [25]:
metadata_values.head(10)

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,AGER_TYP,best-ager typology,0,no classification possible
2,AGER_TYP,best-ager typology,1,passive elderly
3,AGER_TYP,best-ager typology,2,cultural elderly
4,AGER_TYP,best-ager typology,3,experience-driven elderly
5,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,ALTERSKATEGORIE_GROB,age classification through prename analysis,1,< 30 years
7,ALTERSKATEGORIE_GROB,age classification through prename analysis,2,30 - 45 years
8,ALTERSKATEGORIE_GROB,age classification through prename analysis,3,46 - 60 years
9,ALTERSKATEGORIE_GROB,age classification through prename analysis,4,> 60 years


##### Described features

We must check how many features are described in the metadata datasets.

In [26]:
metadata_info_levels.drop_duplicates(subset=['Attribute'])

,Information level,Attribute,Description,Additional notes
0,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,NaN,ANREDE_KZ,gender,NaN
3,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...
...,...,...,...,...
308,Community,ARBEIT,share of unemployed person in the community,NaN
309,NaN,EINWOHNER,inhabitants,NaN
310,NaN,GKZ,standardized community-code,NaN
311,NaN,ORTSGR_KLS9,classified number of inhabitants,NaN


In [27]:
metadata_values.drop_duplicates(subset=['Attribute'])

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
5,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
11,ALTER_HH,main age within the household,0,unknown / no main age detectable
33,ANREDE_KZ,gender,"-1, 0",unknown
36,ANZ_HAUSHALTE_AKTIV,number of households in the building,…,numeric value (typically coded from 1-10)
...,...,...,...,...
2219,WOHNDAUER_2008,length of residence,"-1, 0",unknown
2229,WOHNLAGE,residential-area,-1,unknown
2238,WACHSTUMSGEBIET_NB,growing area (population growth in the last 5 ...,"-1, 0",unknown
2244,W_KEIT_KIND_HH,likelihood of a child present in this household,"-1, 0",unknown


We can see that there are only 313 (in metadata_info_levels dataset) and 314 (in metadata_values dataset) different columns described, but in the AZDIAS dataset, there are 366 demographic columns. We need to check which features have no description in metadata datasets.

In [28]:
metadata_columns = metadata_values.Attribute.unique().tolist()

described_columns = (set(metadata_columns) & set(demographic_columns))
undescribed_columns = (set(metadata_columns) - set(demographic_columns))

print("Number of described demographic columns: ", len(described_columns))
print("Number of undescribed demographic columns: ", len(undescribed_columns))

Number of described demographic columns:  272
Number of undescribed demographic columns:  42


In [29]:
print_df_full(metadata_values[['Attribute', 'Description']].drop_duplicates(subset=['Attribute']))

,Attribute,Description
0,AGER_TYP,best-ager typology
5,ALTERSKATEGORIE_GROB,age classification through prename analysis
11,ALTER_HH,main age within the household
33,ANREDE_KZ,gender
36,ANZ_HAUSHALTE_AKTIV,number of households in the building
37,ANZ_HH_TITEL,number of academic title holder in building
38,ANZ_PERSONEN,number of adult persons in the household
39,ANZ_TITEL,number of professional title holder in household
40,BALLRAUM,distance to next urban centre
48,BIP_FLAG,business-flag indicating companies in the building


### 0.3. Data Preprocessing

##### Nulls normalization

### 0.4. Data Univariate Analysis

#### Remove columns with a high percentage of Null values

#### Remove rows with a high percentage of Null values

#### Classify columns into Numerical or Categorical

#### Transforming Categorical columns into Numerical columns

#### Check ID columns

#### Imputing missing values

### 0.5. Feature Scaling

### 0.6. Build a Function

Build a function that encompasses the stages of **Preprocessing**, **Univariate Analysis** and **Feature Scaling** optionally, in order to apply the same data cleaning process to all datasets used in this project.

#### Check Preprocessing and Univariate Analysis functions

#### Save datasets after Preprocessing and Univariate Analysis steps

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

#### Load clean datasets

#### Scaling data

### 1.1. Principal Component Analysis

#### 1.1.1. Interpreting PCA components

### 1.2. K-Means Clustering

After having reduced the demographic dataset to principal components, we will use a K-Means model to segment the population.

#### 1.2.1. Select optimal K value with AZDIAS_PCA data

##### Choosing Number of Clusters

#### 1.2.2. Fit K-Means object with optimal K and AZDIAS PCA data

#### 1.2.3. Get AZDIAS PCA and CUSTOMERS PCA Clusters

##### AZDIAS Clusters

##### CUSTOMERS Clusters

##### Check results

#### 1.2.4. Get ratio between customers and general population for each Cluster

#### 1.2.5. Analyze Feature Weights in Clusters with highest percentage of customers

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
# mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

#### Check if target column is in dataset

#### Check number of events / no events

### 2.1. Selecting the evaluation metric

### 2.2. Apply Preprocessing and Univariate Analysis steps

#### 2.2.1. Split data for validation and train

### 2.3. Set a benchmark

### 2.4. Evaluate different binary classification estimators

### 2.5. Select the best binary classification estimator

#### 2.5.1. Give the best results after hyperparameter tunning

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter.

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
# mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')
# mailout_test = pd.read_csv(r'./datasets/mailout_test.csv', index_col='Unnamed: 0')